In [46]:
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch
import pandas as pd
import random
import os
import xgboost as xgb

from joblib import dump
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from pytorch_tabnet.tab_model import TabNetRegressor

# CUDA
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(DEVICE.type)

# Fix random seed
seed = 42
torch.manual_seed(seed)
if DEVICE.type == 'cuda':
	torch.cuda.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

cuda


# Pre-processing input data

In [47]:
# train_df and test_df are created in eda notebook
if not os.path.exists('csv/train_df.csv') or not os.path.exists('csv/test_df.csv'):
	print('Please run the eda notebook first')
	exit()
train_df = pd.read_csv('csv/train_df.csv')
test_df = pd.read_csv('csv/test_df.csv')
target = 'total_time_target'
features = test_df.columns.copy().drop(target).drop(['benchmark','brand_raw', 'brand_raw_target','vendor_id_raw', 'arch'])

# log for total_times
train_df[[target, 'total_time']] = np.log1p(train_df[[target, 'total_time']])
test_df[[target, 'total_time']] = np.log1p(test_df[[target, 'total_time']])

X_train = train_df[features]
y_train = train_df[target]

X_test = test_df[features]
y_test = test_df[target]

# Normalize data
x_scaler = MinMaxScaler(feature_range=(0, 1))
X_train = x_scaler.fit_transform(X_train)
X_test = x_scaler.transform(X_test)

y_scaler = MinMaxScaler(feature_range=(0, 1))
y_train = y_scaler.fit_transform(y_train.values.reshape(-1, 1))
y_test = y_scaler.transform(y_test.values.reshape(-1, 1))

## convert to tensor
X_train_t = torch.tensor(X_train, dtype=torch.float32)
X_test_t = torch.tensor(X_test, dtype=torch.float32)
y_train_t = torch.tensor(y_train, dtype=torch.float32)
y_test_t = torch.tensor(y_test, dtype=torch.float32)

if DEVICE.type == 'cuda':
	X_train_t = X_train_t.to(DEVICE)
	y_train_t = y_train_t.to(DEVICE)
	X_test_t = X_test_t.to(DEVICE)
	y_test_t = y_test_t.to(DEVICE)


In [48]:
if not os.path.exists('../models/x_scaler.joblib') or not os.path.exists('../models/y_scaler.joblib'):
	dump(x_scaler, '../models/x_scaler.joblib')
	dump(y_scaler, '../models/y_scaler.joblib')

# Model

In [49]:
class FeedforwardModel(nn.Module):
	def __init__(self, input_dim, dropout=0.1):
		super(FeedforwardModel, self).__init__()
		# layers
		self.model = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Dropout(p=dropout),
			nn.Linear(64, 32),
			nn.ReLU(),
			nn.Dropout(p=dropout),
            nn.Linear(32, 1),
        )
	
	def forward(self, x):
		return self.model(x)
	
	def predict(model, X):
		model.eval()
		with torch.no_grad():
			prediction = model(X)
		return prediction
	
class MCDropoutModel(nn.Module):
	def __init__(self, input_dim, n_hidden_layers, dropout=0.1):
		super(MCDropoutModel, self).__init__()
		if n_hidden_layers < 2:
			raise ValueError("n_hidden_layers must be greater than 1")
		# Input layer
		layers = [
			nn.Linear(input_dim, 200),
			nn.ReLU(),
			nn.Dropout(p=dropout),
		]
		# Hidden layers
		layers.extend([
			nn.Linear(200, 500),
			nn.ReLU(),
			nn.Dropout(p=dropout)
		])
		for _ in range(n_hidden_layers-2):
			layers.extend([
				nn.Linear(500, 500),
				nn.ReLU(),
				nn.Dropout(p=dropout)
			])
		layers.extend([
			nn.Linear(500, 200),
			nn.ReLU(),
			nn.Dropout(p=dropout)
		])
		# Output layer
		layers.append(nn.Linear(200, 2))
		self.model = nn.Sequential(*layers)
	
	def forward(self, x):
		params = self.model(x)
		loc = params[:, 0:1]
		scale =  1e-6 + torch.nn.functional.softplus(0.33 * params[:, 1:2])
		return torch.distributions.Normal(loc, scale)
	
	def predict(self, X):
		self.eval()
		with torch.no_grad():
			predictions = self(X).sample()
		return predictions

In [50]:
def inv_scaling_pyt(y, y_scaler):
	if DEVICE.type == 'cuda':
		return y_scaler.inverse_transform(y.cpu().numpy().reshape(-1, 1))
	else:
		return y_scaler.inverse_transform(y.numpy().reshape(-1, 1))

def inv_scaling(y, y_scaler):
    return y_scaler.inverse_transform(y.reshape(-1, 1))

In [51]:
models_folder = '../models/'
models_path = {
	'tabnet': models_folder + 'tabnet_model.zip',
	'mc_dropout': models_folder + 'mc_dropout_model.pt',
	'feedforward': models_folder + 'feedforward_model.pt',
	'xgboost': models_folder + 'xgboost_model.json'
}

In [52]:
def ensemble_predict(models, X, X_t, y_scaler):
	predictions = []
	prediction = models[0].predict(X_t)
	predictions.append(np.expm1(inv_scaling_pyt(prediction, y_scaler)))
	prediction = models[1].predict(X_t)
	predictions.append(np.expm1(inv_scaling_pyt(prediction, y_scaler)))
	prediction = models[2].predict(X)
	predictions.append(np.expm1(inv_scaling(prediction, y_scaler)))
	prediction = models[3].predict(X)
	predictions.append(np.expm1(inv_scaling(prediction, y_scaler)))
	avg_predictions = np.mean(predictions, axis=0)
	return avg_predictions

# Training

In [53]:
# general models
## TabNet
tabnet = TabNetRegressor()
tabnet.load_model(models_path['tabnet'])
## XGBoost
xgboost = xgb.XGBRegressor()
xgboost.load_model(models_path['xgboost'])

models = [
    torch.load(models_path["feedforward"]).to(DEVICE) if DEVICE.type == 'cuda' else torch.load(models_path["feedforward"], map_location=torch.device('cpu')),
    torch.load(models_path["mc_dropout"]).to(DEVICE) if DEVICE.type == 'cuda' else torch.load(models_path["mc_dropout"], map_location=torch.device('cpu')),
    tabnet,
    xgboost
]

/home/frnk65/resource-prediction-study/venv/lib/python3.10/site-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


In [54]:
preds = ensemble_predict(models, X_test, X_test_t, y_scaler)
y_scaled = np.expm1(inv_scaling(y_test, y_scaler))
rmse = np.sqrt(mean_squared_error(y_scaled, preds))
print(f"RMSE Val loss: {rmse}")

RMSE Val loss: 3.71968791901118


# Describe Validation

In [55]:
def describe_val(model, X, X_t, y, y_scaler):
	min_instance = {"prediction": float('inf'), "actual": 0, "index": 0}
	max_instance = {"prediction": 0, "actual": 0, "index": 0}
	
	predictions = ensemble_predict(model, X, X_t, y_scaler)
	y_scaled = np.expm1(inv_scaling(y, y_scaler))
	index_min = np.argmin(np.abs(predictions - y_scaled))
	min_instance["prediction"] = predictions[index_min].item()
	min_instance["actual"] = y_scaled[index_min].item()
	min_instance["index"] = index_min
	index_max = np.argmax(np.abs(predictions - y_scaled))
	max_instance["prediction"] = predictions[index_max].item()
	max_instance["actual"] = y_scaled[index_max].item()
	max_instance["index"] = index_max

	return min_instance, max_instance, predictions

In [56]:
_, _, predictions = describe_val(models, X_test, X_test_t, y_test, y_scaler)
y_scaled = np.expm1(inv_scaling(y_test, y_scaler))
df = pd.DataFrame(predictions, columns=['predictions'])
df['actual'] = y_scaled
df.to_csv('csv/predictions_ensemble_mcdropout.csv', index=False)

In [57]:
print("Validation set single thread model")
min_instance, max_instance, predictions = describe_val(models, X_test, X_test_t, y_test, y_scaler)
y_scaled = np.expm1(inv_scaling(y_test, y_scaler))
errors = np.abs(predictions - y_scaled)
mean_error = np.mean(errors)
std_error = np.std(errors)

print(f"Mean prediction: {np.mean(predictions)} | Std actual: {np.std(predictions)}")
print(f"Mean actual: {np.mean(y_scaled)} | Std actual: {np.std(y_scaled)}")
print(f"Mean Error: {mean_error} | Std Error: {std_error}")
print("---")
print("Min instance")
print(test_df.iloc[min_instance["index"]])
print(f"Min Prediction: {min_instance['prediction']} | Actual: {min_instance['actual']} | Error: {abs(min_instance['prediction'] - min_instance['actual'])}")
print("---")
print("Max instance")
print(test_df.iloc[max_instance["index"]])
print(f"Max Prediction: {max_instance['prediction']} | Actual: {max_instance['actual']} | Error: {abs(max_instance['prediction'] - max_instance['actual'])}")

Validation set single thread model


Mean prediction: 23.38969612121582 | Std actual: 42.47502136230469
Mean actual: 23.631032844733987 | Std actual: 41.60096829835196
Mean Error: 1.4612992451208908 | Std Error: 3.420626072964837
---
Min instance
total_time                                                         0.122218
total_cpu_usage                                                        0.98
max_ram_usage                                                     13.195312
brand_raw                         Intel(R) Xeon(R) CPU E5-2643 v3 @ 3.40GHz
vendor_id_raw                                                  GenuineIntel
arch                                                                 X86_64
count                                                                    12
l2_cache_size                                                           3.0
l3_cache_size                                                          20.0
l2_cache_line_size                                                      256
l2_cache_associativity        